We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why? Be creative ;)

In [72]:
# Import required libraries
import sqlite3
  
# Connect to SQLite database
conn = sqlite3.connect(r'../data/vivino.db')

# Create a cursor object
cur = conn.cursor()

In [126]:
cur.execute("""
            SELECT *
            FROM wineries 
            WHERE wineries.id = 1574
            ;
            """)
cur.fetchall()

[]

In [128]:

cur.execute("""
            SELECT wines.name, wines.winery_id, wineries.id, wineries.name
            FROM wines
                join wineries on wines.winery_id = wineries.id
            LIMIT 15;
            """)
cur.fetchall()

[('The Armagh Shiraz', 1651, 1651, "Tenuta Tignanello 'Solaia'"),
 ('Colheita Porto', 1652, 1652, 'Tignanello'),
 ('Lion Tamer Napa Valley Red Blend', 1471, 1471, 'Siepi'),
 ('Raut Lagrein', 75712, 75712, 'Corte di Cama Sforzato di Valtellina')]

In [77]:

cur.execute("""
            SELECT  w.name, count(v.id)
            FROM wines w
            JOIN vintages v ON v.wine_id = w.id
            GROUP BY w.name
            LIMIT 10;
            """)
cur.fetchall()


[("'Giulio Ferrari' Riserva del Fondatore", 10),
 ("'Giulio Ferrari' Riserva del Fondatore Rosé", 1),
 ('110 e Lode Merlot', 1),
 ('123 Brunello di Montalcino Riserva', 1),
 ('1694 Classification Red', 1),
 ('25 anni Montefalco Sagrantino', 1),
 ('30 Year Old Tawny Port', 1),
 ('40 Anos Tawny Port', 1),
 ('40 Years Old Port', 1),
 ('40 Years Old Tawny Port', 1)]

In [4]:
cur.execute("""
            SELECT tl.country_code, tl.name,vtr.rank 
            FROM toplists tl
            JOIN vintage_toplists_rankings vtr ON tl.id = vtr.top_list_id
            WHERE tl.name LIKE '%2020%' and tl.name LIKE '%Red%'
            LIMIT 15;
            """)
cur.fetchall()

[('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 2),
 ('global',
  "Vivino's 2020 Wine Style Awards: Southern Rhône Châteauneuf-du-Pape Red",
  1),
 ('global', "Vivino's 2020 Wine Style Awards: Northern Italy Red", 3),
 ('global',
  "Vivino's 2020 Wine Style Awards: Spanish Ribera Del Duero Red",
  3),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 10),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 5),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 8),
 ('global', "Vivino's 2020 Wine Style Awards: Languedoc-Roussillon Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Southern Rhône Red", 1),
 ('global', "Vivino's 2020 Wine Style Awards: Australian Rhône Blend Red", 1)]

In [ ]:
cur.execute("""
            SELECT  DISTINCT country_code
            FROM toplists
            LIMIT 15;
            """)
cur.fetchall()

We have detected that a big cluster of customers like a specific combination of tastes. We have identified a few primary keywords that match this. We would like you to find all the wines that have those keywords. To ensure the accuracy of our selection, ensure that more than 10 users confirmed those keywords. Also, identify the group_name related to those keywords.
- coffee
- toast
- green apple
- cream
- citrus

In [60]:
cur.execute("""
            SELECT w.id, w.name, k.name, kw.group_name, sum(kw.count) 
            FROM wines w
            JOIN keywords_wine kw ON kw.wine_id = w.id
            JOIN keywords k ON k.id = kw.keyword_id
            WHERE k.name in ('coffee','toast','green apple','cream','citrus')
                and kw.count > 10
                and kw.keyword_type = 'primary'
            GROUP BY w.id, w.name, k.name, kw.group_name
            ;
            """)

import csv

result = cur.fetchall()

c = csv.writer(open("../data/temp.csv","w"))
c.writerows(result)




In [76]:
import pandas as pd

cur.execute("""
            SELECT w.name, k.name, kw.group_name, sum(kw.count) 
            FROM wines w
            JOIN keywords_wine kw ON kw.wine_id = w.id
            JOIN keywords k ON k.id = kw.keyword_id
            WHERE k.name in ('coffee','toast','green apple','cream','citrus')
                and kw.count > 10
                and kw.keyword_type = 'primary'
            GROUP BY w.name, k.name, kw.group_name
            ;
            """)

# Put it all to a data frame
sql_data = pd.DataFrame(cur.fetchall(), columns=['name_wine', 'name_keywords', 'group_name', 'count'])

display(sql_data)


,name_wine,name_keywords,group_name,count
0,'Giulio Ferrari' Riserva del Fondatore,citrus,citrus_fruit,66
1,'Giulio Ferrari' Riserva del Fondatore,cream,microbio,72
2,'Giulio Ferrari' Riserva del Fondatore,green apple,tree_fruit,15
3,'Giulio Ferrari' Riserva del Fondatore,toast,non_oak,48
4,25 anni Montefalco Sagrantino,coffee,oak,23
...,...,...,...,...
796,Vénus Blanc de Blancs Brut Nature Champagne Gr...,green apple,tree_fruit,13
797,Vürsù Barbaresco Starderi,coffee,oak,16
798,Vürsù Barolo Campè,coffee,oak,13
799,Yarden Katzrin,coffee,oak,17


In [71]:
import plotly.express as px

